<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/LunarLander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
import glob, os, shutil
from os.path import *
from pathlib import Path
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

contentPth = '/content'
drivePth = '/content/drive'
myDrivePth = '/content/drive/My Drive'
pyFilesPth = '/content/drive/My Drive/PythonFiles'

deletePth = join(contentPth, 'sample_data')
if exists(deletePth):
    shutil.rmtree(deletePth)
else: pass

os.chdir(pyFilesPth)
import ImportDriveFiles
from BashColors import C
from ColabDriveFiles import *
os.chdir(contentPth)
print(f'cwd: {C.Cyan}{Path.cwd()}')

In [ ]:
cdf.listColabFiles(silent=True)
cdf.clearAllCaches()
dirDict=cdf.allDirectoriesDict.copy()
dirPath=dirDict[11]
os.chdir(dirPath)
# print(f'copy files in: {C.IBlue}{os.getcwd()}\n')
cdf.clearAllCaches()
dirFiles=glob('**')
for fil in dirFiles:
    sourcePath=realpath(fil)
    destinationPath=join(cdf.contentPth, fil)
    # print(f'{C.IPurple}{sourcePath}')
    # print(f'{C.IGreen}{destinationPath}')
    shutil.copy2(sourcePath, destinationPath)

In [ ]:
import gym
import numpy as np
from ddpg_torch import Agent
from utils import plot_learning_curve

try: import Box2D
except: 
    %pip install box2d
    import Box2D

if __name__ == '__main__':
    env = gym.make('LunarLanderContinuous-v2')
    agent =Agent(alpha=0.0001, beta=0.001,
                 input_dims=env.observation_space.shape, tau=0.001,
                 n_actions=env.action_space.shape[0], batch_size=64,
                 fc1_dims=400, fc2_dims=300)
    n_games=10 # default 1000
    filename='LunarLander_alpha_' + str(agent.alpha) +  '_beta_' + \
            str(agent.beta) 
    filename='LunarLander'
    print(filename)
    figure_file='plots/' + filename + '.png'
    # print(figure_file)
    best_score=env.reward_range[0] # default -inf
    # print(best_score)
    score_history=[]
    
    for i in range(n_games):
        observation = env.reset()
        done = False
        score = 0
        agent.noise.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            agent.remember(observation, action, reward, observation_, done)
            agent.learn()
            score += reward
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        if avg_score > best_score:
            best_score = avg_score
            agent.save_models()
        # print('episode:', i, 'score: %.1f', % score, 
              # 'average: %.1f', % average_score)
    X = [i+1 for i in range(n_games)]
    plot_learning_curve(x, score_history, figure_file)

In [ ]:
pth='/content/main_ddpg.py'
with open(pth, 'r') as openFile:
    print(openFile.read())